In [1]:
import sys
import numpy as np
import scipy.interpolate as sci
from functools import wraps
sys.path.append("../../TychePlot/")
from Filereader import fileToNpArray,npArrayToFile

In [2]:
name="CsPbBr3_cubes"
file_spectrum="LXV_C_4_8V_25,4C.csv"

In [3]:
wavelengths=np.arange(300,801,1)

In [4]:
def lum_interpolator(lumFunc):
    """
    Prepare an interpolator, that extrapolates or returns zero if the extrapolation is smaller than 0
    """
    interpolator=sci.interp1d(*lumFunc, kind='cubic', fill_value='extrapolate')
    @wraps(interpolator)
    def wrapper(*args,**kwargs):
        return np.maximum(abs(interpolator(*args,**kwargs)*0),interpolator(*args,**kwargs))
    return wrapper

In [5]:
read_options={
                "separator":";",
                "skiplines":82
            }
write_options={
                "separator":"\t",
                #"preString":"bir\n",
                "fileEnding":".sp"
            }

In [6]:
spectrum_array=fileToNpArray(file_spectrum, **read_options)[0]
spectrum_interp=lum_interpolator(spectrum_array.T)
spectrum_interp_data=spectrum_interp(wavelengths).T
new_array_data=np.vstack((wavelengths.T,spectrum_interp_data)).T

In [7]:
npArrayToFile(name,new_array_data,**write_options)